In [4]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary
from sklearn.model_selection import train_test_split

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import collections

In [2]:
# GPU Device
gpu_id = 3
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 3: True


In [5]:
class SiameseNetworkDataset(Dataset):
    
    def __init__(self,imageFolderDataset,transform=None,should_invert=True):
        self.imageFolderDataset = imageFolderDataset    
        self.transform = transform
        self.should_invert = should_invert
        
    def __getitem__(self,index):
        img0_tuple = random.choice(self.imageFolderDataset.imgs)
        #we need to make sure approx 50% of images are in the same class
        should_get_same_class = random.randint(0,1) 
        if should_get_same_class:
            while True:
                #keep looping till the same class image is found
                img1_tuple = random.choice(self.imageFolderDataset.imgs) 
                if img0_tuple[1]==img1_tuple[1]:
                    break
        else:
            while True:
                #keep looping till a different class image is found
                
                img1_tuple = random.choice(self.imageFolderDataset.imgs) 
                if img0_tuple[1] !=img1_tuple[1]:
                    break

        img0 = Image.open(img0_tuple[0])
        img1 = Image.open(img1_tuple[0])
        img0 = img0.convert("L")
        img1 = img1.convert("L")
        
        if self.should_invert:
            img0 = PIL.ImageOps.invert(img0)
            img1 = PIL.ImageOps.invert(img1)

        if self.transform is not None:
            img0 = self.transform(img0)
            img1 = self.transform(img1)
        
        return img0, img1 , torch.from_numpy(np.array([int(img1_tuple[1]!=img0_tuple[1])],dtype=np.float32))
    
    def __len__(self):
        return len(self.imageFolderDataset.imgs)

In [6]:
class ContrastiveLoss(torch.nn.Module):
    """
    Contrastive loss function.
    Based on: http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    """

    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2, keepdim = True)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))


        return loss_contrastive

# Arguments

In [3]:
source_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN_256/'
target_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128/'

In [4]:
pretrained = './log/style1/128/b1/checkpoint.pth.tar'
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b1' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 4000
start_epoch = 0
train_batch = 12
test_batch = 200
lr = 0.1
schedule = [200, 1500, 3000]
momentum = 0
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/style1/128/b1/to_pggan/contrastive' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 4

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# sp
sp_alpha = 0.1
sp_beta = 0.1
fc_name = 'fc.'

# iterative training
feedback = 0
iter_time = [1000, 2000, 2500]

# cutmix
cm_prob = 0.5
cm_prob_init = 0.99
cm_prob_low = 0.1

best_acc = 0

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

# Dataset

In [7]:
train_dir = os.path.join(target_dir, '100_shot_style1')
# source_train_dir = os.path.join(target_dir, '100_shot_style1')
val_target_dir = os.path.join(target_dir, 'validation')
val_source_dir = os.path.join(source_dir, 'validation')

train_aug = transforms.Compose([
#     transforms.RandomAffine(degrees=2, translate=(0.02, 0.02), scale=(0.98, 1.02), shear=2, fillcolor=(124,117,104)),
    transforms.RandomCrop(size),
    
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
#     transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
source_train_loader = DataLoader(datasets.ImageFolder(source_train_dir, transform=val_aug),
                                batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_target_loader = DataLoader(datasets.ImageFolder(val_target_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_source_loader = DataLoader(datasets.ImageFolder(val_source_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [8]:
teacher_model = EfficientNet.from_name(model_name, num_classes=num_classes)
student_model = EfficientNet.from_name(model_name, num_classes=num_classes)

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    teacher_model.load_state_dict(torch.load(pretrained)['state_dict'])
    student_model.load_state_dict(torch.load(pretrained)['state_dict'])

=> using pre-trained model './log/style1/128/b1/checkpoint.pth.tar'


In [9]:
teacher_model.to('cuda')
student_model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in student_model.parameters())/1000000.0))

    Total params: 6.52M


In [10]:
for param in teacher_model.parameters():
    param.requires_grad = False
teacher_model.eval()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): GroupNorm(8, 32, eps=1e-05, affine=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): GroupNorm(8, 32, eps=1e-05, affine=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity()
      )
      (_bn2): GroupNorm(4, 16, ep

In [11]:
teacher_model_weights = {}
for name, param in teacher_model.named_parameters():
    teacher_model_weights[name] = param.detach()

In [12]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

In [13]:
def reg_cls(model):
    l2_cls = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if name.startswith(fc_name):
            l2_cls += 0.5 * torch.norm(param) ** 2
    return l2_cls

def reg_l2sp(model):
    sp_loss = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if not name.startswith(fc_name):
            sp_loss += 0.5 * torch.norm(param - teacher_model_weights[name]) ** 2
    return sp_loss

# Loss

In [14]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(student_model.parameters(), lr=lr, momentum=momentum)
# optimizer = optim.Adam(model.parameters(), weight_decay=0)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=4, total_epoch=50, after_scheduler=scheduler_cosine)

In [15]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    student_model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train Acc.', 'Valid Acc.', 'Source Loss', 'Source ACC.'])

# Train

In [16]:
def train(train_loader, source_train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda):
    student_model.train()
    torch.set_grad_enabled(True)
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    end = time.time()
    
    source_set = []
    for inputs, targets in source_train_loader:
        source_set.append((inputs, targets))
    
    bar = Bar('Processing', max=len(train_loader))
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)
        
        # cutmix
        source_inputs, source_targets = source_set[batch_idx]
        source_inputs, source_targets = source_inputs.cuda(), source_targets.cuda()

        prob_delta = cm_prob_init - cm_prob_low
        prob_step = epoch / (epochs+1) * prob_delta
        lam = cm_prob_init - prob_step

        rand_index = torch.randperm(inputs.size()[0]).cuda()
        st = source_targets[rand_index]
        tt = targets[rand_index]
        rand_index = rand_index[st == tt]

        bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam)
        inputs[rand_index, :, bbx1:bbx2, bby1:bby2] = source_inputs[rand_index, :, bbx1:bbx2, bby1:bby2]
        lam = 1 - ((bbx2 - bbx1)*(bby2 - bby1) / (inputs.size()[-1] * inputs.size()[-2]))
        
        outputs = student_model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(student_model)
        loss_sp = reg_l2sp(student_model)
            
            
        # compute output
        loss = loss_main + sp_alpha*loss_sp + sp_beta*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=batch_idx + 1,
                    size=len(train_loader),
                    data=data_time.val,
                    bt=batch_time.val,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,
                    )
        bar.next()
#         if batch_idx % 10 == 0:
        print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [17]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()
    torch.set_grad_enabled(False)

    end = time.time()
    bar = Bar('Processing', max=len(val_loader))
    for batch_idx, (inputs, targets) in enumerate(val_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs, volatile=True), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(model)
        loss_sp = reg_l2sp(model)
        loss = loss_main + sp_alpha*loss_sp + sp_beta*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:} | top1: {top1:}'.format(
                    batch=batch_idx + 1,
                    size=len(val_loader),
                    data=data_time.avg,
                    bt=batch_time.avg,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,)
        bar.next()
    print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
         batch=batch_idx+1, size=len(val_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [18]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [ ]:
for epoch in range(start_epoch, epochs):
    # teacher feedback
    if epoch in iter_time:
        print("iterative training: feedback {}".format(epoch))
        teacher_model.load_state_dict(student_model.state_dict())
        teacher_model_weights = {}
        for name, param in teacher_model.named_parameters():
            teacher_model_weights[name] = param.detach()
    
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc = train(train_loader, source_train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda)
    
    if epoch % 20 == 0:
        test_loss, test_acc = test(val_target_loader, student_model, criterion, epoch, use_cuda)
        source_loss, source_acc = test(val_source_loader, student_model, criterion, epoch, use_cuda)

    
        logger.append([state['lr'], train_loss, test_loss, train_acc, test_acc, source_loss, source_acc])
        is_best = test_acc > best_acc
        best_acc = max(test_acc, best_acc)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict' : student_model.state_dict(),
            'acc': test_acc,
            'best_acc': best_acc,
            'optimizer': optimizer.state_dict(),
        }, is_best, checkpoint=checkpoint)
    scheduler_warmup.step()


Epoch: [1 | 7000] LR: 0.030000
1/17 Data:12.298 | Batch:29.397 | Total:0:00:20 | ETA:0:05:23 | Loss:1.4609426259994507 | top1:83.33332824707031
2/17 Data:0.002 | Batch:4.417 | Total:0:00:24 | ETA:0:03:05 | Loss:2.1823148131370544 | top1:62.499996185302734
3/17 Data:0.001 | Batch:4.886 | Total:0:00:29 | ETA:0:02:18 | Loss:2.5002040465672812 | top1:63.88888931274414
4/17 Data:0.001 | Batch:3.009 | Total:0:00:32 | ETA:0:01:46 | Loss:3.1693614423274994 | top1:56.25
5/17 Data:0.001 | Batch:5.219 | Total:0:00:37 | ETA:0:01:31 | Loss:3.1472655057907106 | top1:48.333335876464844
6/17 Data:0.001 | Batch:3.980 | Total:0:00:41 | ETA:0:01:17 | Loss:3.1037790179252625 | top1:51.38888931274414
7/17 Data:0.001 | Batch:1.933 | Total:0:00:43 | ETA:0:01:03 | Loss:3.107967325619289 | top1:51.19047546386719
8/17 Data:0.003 | Batch:0.373 | Total:0:00:43 | ETA:0:00:50 | Loss:3.094440594315529 | top1:55.208335876464844
9/17 Data:0.002 | Batch:0.311 | Total:0:00:44 | ETA:0:00:40 | Loss:3.073366893662347 | to

/home/cutz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


161/161 Data:0.002 | Batch:1.793 | Total:0:03:14 | ETA:0:00:00 | Loss:2.946977580447806 | top1:51.872276306152344
39/39 Data:0.446 | Batch:1.198 | Total:0:02:16 | ETA:0:00:00 | Loss:2.664366991091997 | top1:78.93589782714844

Epoch: [2 | 7000] LR: 0.031800
1/17 Data:5.157 | Batch:6.606 | Total:0:00:02 | ETA:0:00:41 | Loss:2.94677996635437 | top1:58.333335876464844
2/17 Data:0.001 | Batch:2.342 | Total:0:00:04 | ETA:0:00:37 | Loss:2.859241008758545 | top1:70.83333587646484
3/17 Data:0.002 | Batch:3.227 | Total:0:00:08 | ETA:0:00:38 | Loss:2.878323793411255 | top1:63.88888931274414
4/17 Data:0.001 | Batch:2.170 | Total:0:00:10 | ETA:0:00:34 | Loss:2.852483868598938 | top1:62.5
5/17 Data:0.002 | Batch:3.159 | Total:0:00:13 | ETA:0:00:33 | Loss:2.8324629783630373 | top1:66.66667175292969
6/17 Data:0.001 | Batch:2.775 | Total:0:00:16 | ETA:0:00:30 | Loss:2.8535533348719277 | top1:62.5
7/17 Data:0.001 | Batch:3.216 | Total:0:00:19 | ETA:0:00:28 | Loss:2.8540761130196706 | top1:60.71428680419

10/17 Data:0.001 | Batch:3.725 | Total:0:00:31 | ETA:0:00:22 | Loss:2.0129921197891236 | top1:67.5
11/17 Data:0.001 | Batch:3.561 | Total:0:00:34 | ETA:0:00:16 | Loss:2.0107646855441006 | top1:67.42424774169922
12/17 Data:0.001 | Batch:4.093 | Total:0:00:38 | ETA:0:00:14 | Loss:2.0173050363858542 | top1:63.88888931274414
13/17 Data:0.002 | Batch:3.813 | Total:0:00:42 | ETA:0:00:12 | Loss:2.006889847608713 | top1:64.1025619506836
14/17 Data:0.002 | Batch:0.396 | Total:0:00:42 | ETA:0:00:09 | Loss:2.0000436902046204 | top1:64.88095092773438
15/17 Data:0.002 | Batch:3.072 | Total:0:00:45 | ETA:0:00:05 | Loss:1.9874385913213095 | top1:65.55555725097656
16/17 Data:0.001 | Batch:2.627 | Total:0:00:48 | ETA:0:00:03 | Loss:1.9912041947245598 | top1:65.10417175292969

Epoch: [7 | 7000] LR: 0.040800
1/17 Data:6.134 | Batch:6.421 | Total:0:00:00 | ETA:0:00:10 | Loss:1.9196550846099854 | top1:66.66667175292969
2/17 Data:0.009 | Batch:3.374 | Total:0:00:03 | ETA:0:00:30 | Loss:1.9227192401885986 | 

5/17 Data:0.001 | Batch:1.421 | Total:0:00:13 | ETA:0:00:32 | Loss:1.252450132369995 | top1:73.33333587646484
6/17 Data:0.001 | Batch:3.788 | Total:0:00:16 | ETA:0:00:32 | Loss:1.2820490797360737 | top1:70.83333587646484
7/17 Data:0.004 | Batch:4.116 | Total:0:00:21 | ETA:0:00:31 | Loss:1.2826955829347884 | top1:71.42857360839844
8/17 Data:0.001 | Batch:4.471 | Total:0:00:25 | ETA:0:00:29 | Loss:1.2857039719820023 | top1:71.875
9/17 Data:0.001 | Batch:3.007 | Total:0:00:28 | ETA:0:00:26 | Loss:1.2836313645044963 | top1:73.14814758300781
10/17 Data:0.001 | Batch:3.591 | Total:0:00:32 | ETA:0:00:23 | Loss:1.2782167553901673 | top1:73.33333587646484
11/17 Data:0.001 | Batch:3.476 | Total:0:00:35 | ETA:0:00:16 | Loss:1.2766067873347888 | top1:73.48484802246094
12/17 Data:0.001 | Batch:3.703 | Total:0:00:39 | ETA:0:00:15 | Loss:1.26393656929334 | top1:74.30555725097656
13/17 Data:0.001 | Batch:4.720 | Total:0:00:44 | ETA:0:00:14 | Loss:1.2625385797940767 | top1:75.64102935791016
14/17 Data:

...

12/17 Data:0.001 | Batch:0.245 | Total:0:00:04 | ETA:0:00:03 | Loss:0.10158051457256079 | top1:98.61111450195312
13/17 Data:0.001 | Batch:0.312 | Total:0:00:04 | ETA:0:00:02 | Loss:0.11206217024188775 | top1:98.0769271850586
14/17 Data:0.002 | Batch:0.476 | Total:0:00:05 | ETA:0:00:01 | Loss:0.11699499216462884 | top1:97.61904907226562
15/17 Data:0.001 | Batch:0.661 | Total:0:00:06 | ETA:0:00:01 | Loss:0.11517756208777427 | top1:97.77777862548828
16/17 Data:0.002 | Batch:0.740 | Total:0:00:06 | ETA:0:00:01 | Loss:0.11076413653790951 | top1:97.91667175292969

Epoch: [1765 | 7000] LR: 0.001031
1/17 Data:3.468 | Batch:4.106 | Total:0:00:01 | ETA:0:00:32 | Loss:0.04207320511341095 | top1:100.0
2/17 Data:0.001 | Batch:0.432 | Total:0:00:02 | ETA:0:00:19 | Loss:0.043311720713973045 | top1:100.0
3/17 Data:0.003 | Batch:0.363 | Total:0:00:02 | ETA:0:00:14 | Loss:0.10152675583958626 | top1:94.44444274902344
4/17 Data:0.002 | Batch:0.487 | Total:0:00:03 | ETA:0:00:11 | Loss:0.09669920895248652 |

6/17 Data:0.002 | Batch:0.657 | Total:0:00:03 | ETA:0:00:08 | Loss:0.06633188389241695 | top1:98.61111450195312
7/17 Data:0.001 | Batch:0.441 | Total:0:00:04 | ETA:0:00:07 | Loss:0.06981776282191277 | top1:98.80952453613281
8/17 Data:0.001 | Batch:0.545 | Total:0:00:04 | ETA:0:00:06 | Loss:0.06833452731370926 | top1:98.95833587646484
9/17 Data:0.001 | Batch:0.290 | Total:0:00:05 | ETA:0:00:05 | Loss:0.06684257586797078 | top1:99.0740737915039
10/17 Data:0.002 | Batch:0.303 | Total:0:00:05 | ETA:0:00:04 | Loss:0.06893386021256447 | top1:99.16667175292969
11/17 Data:0.002 | Batch:0.513 | Total:0:00:05 | ETA:0:00:04 | Loss:0.06677644632079384 | top1:99.24242401123047
12/17 Data:0.002 | Batch:0.272 | Total:0:00:06 | ETA:0:00:03 | Loss:0.07951623201370239 | top1:98.61111450195312
13/17 Data:0.002 | Batch:0.326 | Total:0:00:06 | ETA:0:00:02 | Loss:0.07657605925431618 | top1:98.71794891357422
14/17 Data:0.009 | Batch:0.683 | Total:0:00:07 | ETA:0:00:02 | Loss:0.07436138018965721 | top1:98.809

16/17 Data:0.002 | Batch:0.434 | Total:0:00:08 | ETA:0:00:01 | Loss:0.12134777009487152 | top1:97.91667175292969

Epoch: [1774 | 7000] LR: 0.001030
1/17 Data:1.798 | Batch:2.336 | Total:0:00:01 | ETA:0:00:20 | Loss:0.11186923086643219 | top1:91.66667175292969
2/17 Data:0.003 | Batch:0.291 | Total:0:00:01 | ETA:0:00:12 | Loss:0.07782483659684658 | top1:95.83333587646484
3/17 Data:0.001 | Batch:0.424 | Total:0:00:01 | ETA:0:00:09 | Loss:0.07242347796758015 | top1:97.22222137451172
4/17 Data:0.001 | Batch:0.359 | Total:0:00:02 | ETA:0:00:08 | Loss:0.07952678576111794 | top1:95.83333587646484
5/17 Data:0.003 | Batch:0.480 | Total:0:00:02 | ETA:0:00:07 | Loss:0.13778713643550872 | top1:95.00000762939453
6/17 Data:0.002 | Batch:0.413 | Total:0:00:03 | ETA:0:00:06 | Loss:0.20947660754124323 | top1:94.44444274902344
7/17 Data:0.001 | Batch:0.557 | Total:0:00:03 | ETA:0:00:06 | Loss:0.18557199356811388 | top1:95.23809814453125
8/17 Data:0.002 | Batch:0.308 | Total:0:00:04 | ETA:0:00:05 | Loss:0

8/17 Data:0.003 | Batch:0.595 | Total:0:00:06 | ETA:0:00:08 | Loss:0.06304720835760236 | top1:98.95833587646484
9/17 Data:0.001 | Batch:0.711 | Total:0:00:06 | ETA:0:00:07 | Loss:0.07099178805947304 | top1:98.14814758300781
10/17 Data:0.002 | Batch:0.465 | Total:0:00:07 | ETA:0:00:06 | Loss:0.07064782418310642 | top1:98.33333587646484
11/17 Data:0.001 | Batch:0.728 | Total:0:00:08 | ETA:0:00:05 | Loss:0.07153257355093956 | top1:98.48484802246094
12/17 Data:0.002 | Batch:0.807 | Total:0:00:08 | ETA:0:00:04 | Loss:0.09079507148514192 | top1:97.91666412353516
13/17 Data:0.002 | Batch:0.646 | Total:0:00:09 | ETA:0:00:03 | Loss:0.0882298370393423 | top1:98.0769271850586
14/17 Data:0.001 | Batch:0.514 | Total:0:00:10 | ETA:0:00:02 | Loss:0.0851535193089928 | top1:98.21428680419922
15/17 Data:0.001 | Batch:0.521 | Total:0:00:10 | ETA:0:00:02 | Loss:0.08273630142211914 | top1:98.33333587646484
16/17 Data:0.003 | Batch:0.595 | Total:0:00:11 | ETA:0:00:01 | Loss:0.08141784835606813 | top1:98.437

16/17 Data:0.001 | Batch:0.599 | Total:0:00:08 | ETA:0:00:01 | Loss:0.07118507288396358 | top1:99.47917175292969

Epoch: [1783 | 7000] LR: 0.001028
1/17 Data:1.914 | Batch:2.762 | Total:0:00:01 | ETA:0:00:25 | Loss:0.24835917353630066 | top1:91.66667175292969
2/17 Data:0.004 | Batch:0.523 | Total:0:00:02 | ETA:0:00:16 | Loss:0.14951330795884132 | top1:95.83333587646484
3/17 Data:0.001 | Batch:0.566 | Total:0:00:02 | ETA:0:00:13 | Loss:0.11782243226965268 | top1:97.22222137451172
4/17 Data:0.002 | Batch:0.637 | Total:0:00:03 | ETA:0:00:11 | Loss:0.10508726071566343 | top1:97.91667175292969
5/17 Data:0.002 | Batch:0.581 | Total:0:00:03 | ETA:0:00:10 | Loss:0.10382808074355125 | top1:98.33333587646484
6/17 Data:0.001 | Batch:0.337 | Total:0:00:04 | ETA:0:00:08 | Loss:0.10754226831098397 | top1:97.22222137451172
7/17 Data:0.002 | Batch:0.602 | Total:0:00:04 | ETA:0:00:07 | Loss:0.10071023819702012 | top1:97.61904907226562
8/17 Data:0.003 | Batch:0.431 | Total:0:00:05 | ETA:0:00:06 | Loss:0